In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

# Import libraries and data


In [3]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)

In [6]:
df = pd.read_csv('/content/Wongnai_reviews_copper_beyond_buffet - Sheet1.csv')

In [7]:
df

,Review
0,สุดยอดบุฟเฟต์ที่ดีเสมอต้นเสมอปลาย เป็นBuffetใน...
1,เป็นร้านบุฟเฟ่ต์นานาชติที่ฮอตฮิตตลิดกาล ครั้งน...
2,ของเขาดีจริง ต้องบอกว่ามีความคุ้มทุกบาททุกสตาง...
3,อาหารอร่อย กุ้งเผาสดดี น้ำจิ้มซีฟู้ดอร่อย เตี๋...
4,เป็นร้านบุฟเฟ่ต์นานาชาติอันดับ 1 จากหลายสถาบัน...
5,สวย โปร่ง โล่ง สบาย Luxury มากๆ
6,เราค่อนข้างประทับใจกับระบบการจัดการ ความขยันตั...
7,ผมบอกเลยว่าสุดๆจริงๆ วิธีการสั่งอาหาร แบบใหม่ ...
8,เราว่าโอเคมาก ประทับใจ น้องพนักงานน่ารักที่สุด...


# Tokenize words with pythainlp

In [28]:

stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(',')']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine = 'newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [29]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [30]:
df

,Review,Review_tokenized
0,สุดยอดบุฟเฟต์ที่ดีเสมอต้นเสมอปลาย เป็นBuffetใน...,"สุดยอด,บุฟเฟต์,ดี,เสมอต้นเสมอปลาย,Buffet,ราคา,..."
1,เป็นร้านบุฟเฟ่ต์นานาชติที่ฮอตฮิตตลิดกาล ครั้งน...,"บุฟเฟ่ต์,นานา,ช,ติ,ฮอต,ฮิต,ต,ลิด,กาล,จอง,ราคา,..."
2,ของเขาดีจริง ต้องบอกว่ามีความคุ้มทุกบาททุกสตาง...,"ดีจริง,บอ,คุ้ม,บาท,สตางค์,อาหาร,จาน,ดีเทล,โซน,..."
3,อาหารอร่อย กุ้งเผาสดดี น้ำจิ้มซีฟู้ดอร่อย เตี๋...,"อาหาร,อร่อย,กุ้ง,เผา,สด,ดี,น้ำจิ้ม,ซี,ฟู้ด,อร่..."
4,เป็นร้านบุฟเฟ่ต์นานาชาติอันดับ 1 จากหลายสถาบัน...,"บุฟเฟ่ต์,นานาชาติ,อันดับ,1,สถาบัน,จ่าย,1,000,...."
5,สวย โปร่ง โล่ง สบาย Luxury มากๆ,"สวย,โปร่ง,โล่ง,Luxury"
6,เราค่อนข้างประทับใจกับระบบการจัดการ ความขยันตั...,"ประทับใจ,ระบบ,ขยัน,ตั้งใจ,บริการ,ความสุภาพ,พนั..."
7,ผมบอกเลยว่าสุดๆจริงๆ วิธีการสั่งอาหาร แบบใหม่ ...,"ผม,วิธีการ,สั่งอาหาร,แบ่ง,โซน,ฝั่ง,เครื่องดื่ม..."
8,เราว่าโอเคมาก ประทับใจ น้องพนักงานน่ารักที่สุด...,"โอเค,ประทับใจ,น้อง,พนักงาน,น่ารัก,รีวิว,เมนู,ข..."


# Create dictionary


In [33]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [35]:
print(dictionary.token2id.keys())

dict_keys(['Buffet', 'คุ้มค่า', 'ดี', 'บุฟเฟต์', 'ผิดหวัง', 'พัน', 'ราคา', 'ลอง', 'วจะ', 'สุดยอด', 'อาหาร', 'เสมอต้นเสมอปลาย', 'เสิร์ฟ', 'แนะนำ', 'แล้', '', '1', '300', 'กาล', 'คา', 'คุ้ม', 'จอง', 'ช', 'ญี่ปุ่น', 'ต', 'ตัวเรา', 'ติ', 'ทาน', 'นรา', 'นานา', 'บุฟเฟ่ต์', 'ฝรั่ง', 'ลิด', 'ฮอต', 'ฮิต', 'เป้', 'เลือก', 'โปรโมชั่น', 'ไทย', 'Basque', 'Cheesecake', 'Paparch', 'กิน', 'งี้', 'จาน', 'ดีจริง', 'ดีย์', 'ดีเทล', 'นุ่ม', 'บอ', 'บาท', 'รมควัน', 'ลิ้น', 'วัว', 'สตางค์', 'หอม', 'เนื้อ', 'เหนียว', 'แล้วก็', 'โซน', 'กุ้ง', 'ขนมจีน', 'ขนมหวาน', 'ขิต', 'ซี', 'ตอน', 'น้ำจิ้ม', 'น้ำยา', 'ปู', 'พะอืดพะอม', 'ฟู้ด', 'สด', 'อร่อย', 'ออกจาก', 'อาเจียน', 'เก่า', 'เตี๋ยว', 'เผา', 'เรือ', 'เหมือน', '-', '.', '000', 'ของหวาน', 'คนไทย', 'ความสะอาด', 'คอ', 'จีน', 'จ่าย', 'ซ้ำ', 'ตะวันตก', 'ตัวเลือก', 'ต้นตำรับ', 'ถูกคอ', 'ถูกปาก', 'ทำได้', 'นานาชาติ', 'บริการ', 'ปรุง', 'ฟิวชั่น', 'รสจัด', 'รสชาติ', 'ล', 'สถานที่', 'สถาบัน', 'สักนิด', 'สำหรับ', 'หลากหลาย', 'อันดับ', 'เชฟ', 'เพียบ', 'เมนู', 'เรื่อง', 'แลบ',

In [37]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [52]:
# topic modeling
num_topics = 10
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes throught documents
iterations = 50
eval_every = 1 # don't evalute model perplexity, takes too much time.

# make a index to word dictionary.
temp = dictionary[0] # this is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, alpha ='auto', eta='auto', iterations=iterations, num_topics=num_topics, passes=passes, eval_every=eval_every)

CPU times: user 165 ms, sys: 0 ns, total: 165 ms
Wall time: 267 ms


# Visualizing results


In [53]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.074048 -0.175041       1        1  38.855654
6      0.187790  0.006996       2        1  21.950317
0     -0.163099  0.098245       3        1  19.629573
8      0.015931  0.057482       4        1  12.005580
4      0.032654  0.025567       5        1   7.151441
1      0.000056 -0.002667       6        1   0.085897
9      0.000250 -0.002643       7        1   0.082730
7      0.000151 -0.002653       8        1   0.082241
2      0.000158 -0.002643       9        1   0.078283
3      0.000158 -0.002643      10        1   0.078283, topic_info=        Term      Freq     Total Category  logprob  loglift
72     อร่อย  5.000000  5.000000  Default  30.0000  30.0000
10     อาหาร  6.000000  6.000000  Default  29.0000  29.0000
6       ราคา  3.000000  3.000000  Default  28.0000  28.0000
112   เรื่อง  1.000000  1.000000  Default  27.0000  27.0000
71        สด  1.000000  1.000000  Default  26.0000  26.0000
..       ...       ...       ...      ...      ...      ...
23   ญี่ปุ่น  0.000975  1.871196  Topic10  -5.1874  -0.4068
24         ต  0.000975  1.045357  Topic10  -5.1874   0.1754
25    ตัวเรา  0.000975  1.045344  Topic10  -5.1874   0.1754
26        ติ  0.000975  1.045346  Topic10  -5.1874   0.1754
27       ทาน  0.000975  1.045366  Topic10  -5.1874   0.1754

[531 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
15        3  0.570629      
80        1  0.859023     -
81        1  0.858993     .
82        1  0.858996   000
16        1  0.741516     1
...     ...       ...   ...
38        1  0.534395   ไทย
38        3  0.534395   ไทย
177       2  0.934626   ไป๊
131       5  1.236576  ไลน์
178       2  0.934613   ไหว

[196 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 7, 1, 9, 5, 2, 10, 8, 3, 4])

# Predict Topics

In [54]:
model.show_topics(1)

[(5,
  '0.039*"อาหาร" + 0.030*"อร่อย" + 0.020*"กิน" + 0.020*"ดี" + 0.020*"สด" + 0.020*"1" + 0.020*"เรื่อง" + 0.010*"เครื่องดื่ม" + 0.010*"ปู" + 0.010*"พะอืดพะอม"')]

In [60]:
df['topic'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [61]:
df

,Review,Review_tokenized,topic,score
0,สุดยอดบุฟเฟต์ที่ดีเสมอต้นเสมอปลาย เป็นBuffetใน...,"สุดยอด,บุฟเฟต์,ดี,เสมอต้นเสมอปลาย,Buffet,ราคา,...",0,0.981403
1,เป็นร้านบุฟเฟ่ต์นานาชติที่ฮอตฮิตตลิดกาล ครั้งน...,"บุฟเฟ่ต์,นานา,ช,ติ,ฮอต,ฮิต,ต,ลิด,กาล,จอง,ราคา,...",0,0.989272
2,ของเขาดีจริง ต้องบอกว่ามีความคุ้มทุกบาททุกสตาง...,"ดีจริง,บอ,คุ้ม,บาท,สตางค์,อาหาร,จาน,ดีเทล,โซน,...",8,0.986859
3,อาหารอร่อย กุ้งเผาสดดี น้ำจิ้มซีฟู้ดอร่อย เตี๋...,"อาหาร,อร่อย,กุ้ง,เผา,สด,ดี,น้ำจิ้ม,ซี,ฟู้ด,อร่...",5,0.989451
4,เป็นร้านบุฟเฟ่ต์นานาชาติอันดับ 1 จากหลายสถาบัน...,"บุฟเฟ่ต์,นานาชาติ,อันดับ,1,สถาบัน,จ่าย,1,000,....",5,0.993731
5,สวย โปร่ง โล่ง สบาย Luxury มากๆ,"สวย,โปร่ง,โล่ง,Luxury",0,0.012490
6,เราค่อนข้างประทับใจกับระบบการจัดการ ความขยันตั...,"ประทับใจ,ระบบ,ขยัน,ตั้งใจ,บริการ,ความสุภาพ,พนั...",4,0.980150
7,ผมบอกเลยว่าสุดๆจริงๆ วิธีการสั่งอาหาร แบบใหม่ ...,"ผม,วิธีการ,สั่งอาหาร,แบ่ง,โซน,ฝั่ง,เครื่องดื่ม...",5,0.983981
8,เราว่าโอเคมาก ประทับใจ น้องพนักงานน่ารักที่สุด...,"โอเค,ประทับใจ,น้อง,พนักงาน,น่ารัก,รีวิว,เมนู,ข...",6,0.993437
